In [1]:
import pandas as pd
import re
from tqdm import tqdm_notebook

prefix = 'data/'

In [2]:
df_all = pd.read_csv(prefix +"/spam_dataset_with_subject.csv",index_col=0)

In [3]:
df_all = df_all.drop_duplicates()
df_all.isna().sum()
df_all.dropna(subset=['text_in_img', 'text_in_html'], how='all', inplace = True)


In [4]:
df_all.shape

(11182, 10)

In [5]:
df_all = df_all.fillna("")

In [6]:
df_all.head()

,index,account_id,agent_id,sender,text_in_html,text_in_img,img_text_ratio,label,id_pair,subject
0,0,1129379786086,1132572860140,Wayside Recovery Center,RSVP Today and join us for a morning of empowe...,,0.0,0,"1129379786086,1132572860140",Only 3 weeks away! Empower ME Breakfast
1,1,1119086518675,1132430885433,Hemophilia Foundation of Greater Florida,Walk As Your Favorite Superhero CAN'T MAKE IT ...,,0.0,0,"1119086518675,1132430885433",Tampa Sunday Funday
2,2,1129108515095,1132943659666,Post and Courier,"Good morning, here are your Clemson sports hea...",,0.0,0,"1129108515095,1132943659666",Clemson basketball team wins gold medal for US...
3,3,1129108515095,1132938328084,Post and Courier Advertising Partners,Carmella's advanced in 5 Charleston's Choice c...,,0.0,0,"1129108515095,1132938328084",Indulge your sweet tooth. Vote for Carmella's
4,4,1103447982417,1132525777889,Bravo Troy Ohio,Easter Brunch Santa Claus doesn't hold a candl...,,0.0,0,"1103447982417,1132525777889","The GOOD NEWS of Troy, Ohio - 4/21/2019"


In [12]:
import re
def remove_emoji(s):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', s)

In [13]:
def string_preprocess(r,c1,c2,c3):
    s=r[c1]+' '+r[c2]
    if len(r[c2]) < 200:
        s+=' '+r[c3]
    s= re.sub('\s+', ' ', s).strip()
    if len(s) == 0:
        return ""
    return remove_emoji(s)

df_all['text'] = df_all.apply (lambda row: string_preprocess(row,'subject', 'text_in_html', 'text_in_img'), axis=1)


In [14]:
df_bert = pd.DataFrame({
    'id':range(len(df_all)),
    'label':df_all['label'],
    'alpha':['a']*df_all.shape[0],
    'text': df_all['text'] 
})

In [15]:
df_bert.isna().sum()

id       0
label    0
alpha    0
text     0
dtype: int64

In [16]:
df_bert.shape

(11182, 4)

In [17]:
df_bert['text']

0        Only 3 weeks away! Empower ME Breakfast RSVP T...
1        Tampa Sunday Funday Walk As Your Favorite Supe...
2        Clemson basketball team wins gold medal for US...
3        Indulge your sweet tooth. Vote for Carmella's ...
4        The GOOD NEWS of Troy, Ohio - 4/21/2019 Easter...
5        MORE GOOD NEWS of Troy, Ohio - 7/7/2019 Corky ...
6        AUCTION-Register Now For Litz Crane & Rigging!...
7        How to Lower Your Risk of Having More Breakout...
8        Psyche Institute, Rainbow Reiki Training, Pet ...
9        July - Employee Newsletter Employee of the Mon...
12       Reminder Camp Hours Reminder : Summer Camp Hou...
13       The latest news for you about Field Trip & Kon...
14       One big boy moves out, a big girl moves in! Mo...
15       2019 Mid-Year Market Outlook Conference Call M...
16       Alcatel-Lucent 8058s IP Phone Price Decrease 8...
17       Email Exclusive: Christmas in July Sale! Super...
18       Is dirt bike insurance the same as motorcycle .

In [18]:
X = df_bert[['id','alpha','text']]
y = df_bert['label']

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

train_df = pd.DataFrame({
    'id': X_train['id'],
    'label':y_train,
    'alpha': ['a']*X_train.shape[0],
    'text': X_train['text']
})


test_df = pd.DataFrame({
    'id': X_test['id'],
    'label':y_test,
    'alpha': ['a']*X_test.shape[0],
    'text': X_test['text']
})

In [20]:
test_df.shape

(2237, 4)

In [21]:
train_df.shape

(8945, 4)

In [24]:
train_df.to_csv(prefix+'train.tsv', sep='\t', index=False, header=False)
test_df.to_csv(prefix+'dev.tsv', sep='\t', index=False, header=False)

In [26]:
!aws  s3 cp data/dev.tsv s3://sagemaker-us-east-1-665028136136/compliance-data/batch2/
!aws  s3 cp data/train.tsv s3://sagemaker-us-east-1-665028136136/compliance-data/batch2/

upload: data/dev.tsv to s3://sagemaker-us-east-1-665028136136/compliance-data/batch2/dev.tsv
upload: data/train.tsv to s3://sagemaker-us-east-1-665028136136/compliance-data/batch2/train.tsv
